## Introduction

**polytof** optimizes quantum circuits over the {H, CNOT, Toffoli} gate set.

The toolchain works with **CNOT+CCZ circuits**: every {H, CNOT, Toffoli} circuit is first compiled into this form by replacing internal Hadamard gates with ancilla-based gadgets (following the [AlphaTensor-Quantum](https://arxiv.org/abs/2402.14396) approach, where the measurement in the gadget is assumed to return zero — this does not change the non-Clifford behaviour of the circuit).

The non-Clifford part of a CNOT+CCZ circuit is captured by a **phase polynomial**, which we represent as a symmetric cubic tensor over GF(2). Optimizing this tensor then translates directly into circuit optimization.

**Three-stage optimization pipeline:**

1. **BCO** (Basis Change Optimization) — minimize the number of nonzero entries (nnz) via GF(2) transvections
2. **CPD** (Canonical Polyadic Decomposition) — find a low-rank decomposition; the CP rank equals the **Toffoli count**
3. **Waring** — convert to a Waring decomposition and optimize via FastTODD; the Waring rank equals the **T-count**

## Prerequisites

- **C++ compiler** with C++20 support: GCC 12+ or Clang 15+
- **Python 3.8+** with NumPy and Numba
- **pthreads** support (standard on Linux/macOS; on Windows use MSYS2/MinGW)

> Commands below use Unix-style paths (`bin/bco`). On Windows, use backslashes: `bin\bco`.

## Compilation

All programs are compiled directly with `g++`. No build system (CMake/Make) is needed.

The `VEC_WORDS` define controls the bitvector width:
- `VEC_WORDS=1` — supports tensors with n ≤ 64 (covers most circuit benchmarks)
- `VEC_WORDS=2` — n ≤ 128
- `VEC_WORDS=6` — n ≤ 384 (needed for the largest benchmarks like Ham15-high or Mod_Adder_1024)

In [1]:
import os
os.chdir(os.path.join(os.path.dirname(os.getcwd()), '') or '..')  # ensure we're in the repo root
os.makedirs('bin', exist_ok=True)

In [ ]:
# Circuit compiler
!g++ -Ofast -std=c++20 -march=native -s -pthread -I third_party -I src src/compile.cpp -o bin/compile

# BCO (Basis Change Optimization)
!g++ -Ofast -std=c++20 -march=native -s -pthread -I third_party -I src src/bco.cpp -o bin/bco

# CPD — three variants for different tensor sizes
!g++ -D VEC_WORDS=1 -Ofast -std=c++20 -march=native -s -pthread -I third_party -I src src/cpd.cpp -o bin/topp1
!g++ -D VEC_WORDS=2 -Ofast -std=c++20 -march=native -s -pthread -I third_party -I src src/cpd.cpp -o bin/topp2
!g++ -D VEC_WORDS=6 -Ofast -std=c++20 -march=native -s -pthread -I third_party -I src src/cpd.cpp -o bin/topp6

# Waring — matching variants
!g++ -D VEC_WORDS=1 -Ofast -std=c++20 -march=native -s -pthread -I third_party -I src src/waring.cpp -o bin/waring1
!g++ -D VEC_WORDS=6 -Ofast -std=c++20 -march=native -s -pthread -I third_party -I src src/waring.cpp -o bin/waring6

## End-to-End Example: Tof_3

We walk through the full pipeline on a small circuit: **Tof_3** (a 3-qubit Toffoli gate decomposition into CNOT+CCZ).

The circuit uses 5 qubits (4 logical + 1 ancilla) and consists of Hadamard gates sandwiching CCZ gates.

### Step 1: Write the circuit file

In [3]:
import os
os.makedirs('data/circuits/raw', exist_ok=True)

circuit = """.v 1 2 3 4 5
.i 1 2 3 4

BEGIN
H 5
Z 1 2 5
H 5
H 4
Z 3 5 4
H 4
H 5
Z 1 2 5
H 5
END
"""

with open('data/circuits/raw/0820.qc', 'w') as f:
    f.write(circuit)

print('Written data/circuits/raw/0820.qc')

Written data/circuits/raw/0820.qc


### Step 2: Compile the circuit

The compiler parses the `.qc` file, replaces internal Hadamard gates with ancilla gadgets, extracts the phase polynomial, and saves the corresponding symmetric tensor.

- `--idx` tells it to also save the tensor in sparse triple format to `data/tensors/`

In [4]:
!bin/compile 820 --idx

0820
  input:  5 qubits, 9 gates, T=21, H=4
  merge:  T=15
  output: 7 qubits (2 ancillas), P=[15x7]
  tensor: 3 nnz (|f1|=0, |f2|=0, |f3|=3)


### Step 3: BCO — minimize nonzero entries

BCO applies GF(2) transvections (basis changes x_i ← x_i ⊕ x_j) to reduce the tensor's nnz.

The output tensor gets ID = input + 1000 (i.e., 0820 → 1820).

In [5]:
!bin/bco 820 -b 1 --save --verify -v

Loading tensor 0820
n=7 nnz=3 beam=1 threads=8

  Iter 1: nnz=2 * (0.00s)
  Iter 2: nnz=2 (0.00s)
  Iter 3: nnz=2 (0.00s)
  Iter 4: nnz=2 (0.00s)
  Iter 5: nnz=2 (0.00s)
  Iter 6: nnz=2 (0.00s)

=== BCO 0820 -> 1820 ===
Initial:   3
Final:     2 (ok)
Reduction: 1 (33.3%)
Iters:     6
Time:      0.00s
Tensor:    /home/htc/kkhoruzhii/SCRATCH/polytof/data/tensors/1820.npy
Transform: /home/htc/kkhoruzhii/SCRATCH/polytof/data/transform/0820-1820.npy


### Step 4: SGE — compute CP rank (Toffoli count)

Symmetric Greedy Elimination (SGE) finds a CP decomposition of the BCO-optimized tensor.

The CP rank equals the Toffoli count of the optimized circuit.

In [6]:
!bin/topp1 1820 --sge --save --verify

=== CPD ===
Tensor: 1820
Path: data/tensors/1820.npy
Dims: 7x7x7, nnz: 2
Seed: 7206231189773208708
Mode: sge(beam=1)

Trivial rank: 2

=== Preprocess ===
SGE (beam=1): 2 -> 2 (iters=0) [0.000s] ok

=== Result ===
Best rank: 2
Pool size: 1
Saved: /home/htc/kkhoruzhii/SCRATCH/polytof/data/cpd/topp/1820-00002.npy


### Step 5: Waring decomposition (T-count)

Each CP rank-1 term expands into up to 7 parity vectors (Waring rank-1 terms). FastTODD then optimizes the T-count by cancelling redundant parities.

In [7]:
!bin/waring1 1820 --cpd --save --verify

Tensor 1820: n=7, nnz=2
Using minimum available CPD rank: 2
Loaded 1 CPD schemes from 1 file(s)
[1/1] 14 -> 13 (nnz=30, t=0.00s) *

WARING 1820
  Mode    cpd (cpd-rank=2, 1 schemes)
  Method  FastTODD, 4 threads
  Result  13.0 +- 0.0 (min 13, max 13, nnz 30)
  Time    0.00s
  Saved   /home/htc/kkhoruzhii/SCRATCH/polytof/data/waring/1820-00013.npy
  Verify  PASS


### Summary

The full pipeline for Tof_3:

| Step | Tool | Input | Output |
|------|------|-------|--------|
| Compile | `bin/compile` | `.qc` file | tensor (n, nnz, T-count) |
| BCO | `bin/bco` | tensor 0820 | tensor 1820 (reduced nnz) |
| CPD | `bin/topp1 --sge` | tensor 1820 | CP rank = Toffoli count |
| Waring | `bin/waring1 --cpd` | CPD of 1820 | Waring rank = T-count |

## BCO for Circuit Benchmarks

We run BCO with beam width 1 (greedy, fast) on all circuit tensors from the 01xx (AlphaTensor-Quantum) and 08xx (Todd et al.) series.

Tensors are expected to already exist in `data/tensors/`.

In [8]:
import glob
import subprocess
import re

# Collect circuit tensor IDs (01xx and 08xx)
tensor_files = sorted(
    glob.glob('data/tensors/01??.npy') + glob.glob('data/tensors/08??.npy')
)
tensor_ids = [int(f.replace("\\", "/").split('/')[-1].split('.')[0]) for f in tensor_files]
tensor_ids = [j for j in tensor_ids if j % 100 != 76] # skip qft_4

print(f'Found {len(tensor_ids)} circuit tensors')
print(f'01xx: {sum(1 for t in tensor_ids if 100 <= t < 200)}')
print(f'08xx: {sum(1 for t in tensor_ids if 800 <= t < 900)}')

Found 94 circuit tensors
01xx: 62
08xx: 32


In [9]:
bco_results = []

for tid in tensor_ids:
    result = subprocess.run(
        ['bin/bco', str(tid), '-b', '1', '--save', '--verify'],
        capture_output=True, text=True
    )
    output = result.stdout

    # Parse output
    m_n = re.search(r'n=(\d+)', output)
    m_init = re.search(r'Initial:\s+(\d+)', output)
    m_final = re.search(r'Final:\s+(\d+)', output)

    if m_n and m_init and m_final:
        bco_results.append({
            'id': tid,
            'n': int(m_n.group(1)),
            'nnz_before': int(m_init.group(1)),
            'nnz_after': int(m_final.group(1)),
        })
        print(f'  {tid:04d}: n={bco_results[-1]["n"]:3d}  nnz {bco_results[-1]["nnz_before"]:5d} -> {bco_results[-1]["nnz_after"]:5d}')
    else:
        print(f'  {tid:04d}: FAILED')
        print(output)

print(f'\nDone: {len(bco_results)}/{len(tensor_ids)} tensors processed')

  0101: n=  8  nnz     7 ->     2
  0102: n= 14  nnz    28 ->     4
  0103: n= 20  nnz    64 ->     6
  0104: n= 50  nnz   591 ->    31
  0105: n= 21  nnz    61 ->     8
  0106: n= 42  nnz   335 ->    14
  0108: n= 42  nnz   891 ->    19
  0111: n= 27  nnz   119 ->    14
  0112: n=  5  nnz     5 ->     1
  0114: n= 11  nnz    25 ->     3
  0115: n= 28  nnz   365 ->    18
  0117: n= 42  nnz  1003 ->    16
  0119: n= 24  nnz   101 ->     6
  0120: n=  7  nnz    11 ->     2
  0121: n= 11  nnz    30 ->     8
  0122: n= 15  nnz    63 ->     4
  0123: n= 35  nnz   582 ->    27
  0124: n= 14  nnz    78 ->     3
  0125: n=  6  nnz    11 ->     5
  0126: n=  9  nnz    25 ->    12
  0127: n= 12  nnz    30 ->    21
  0128: n= 15  nnz    79 ->    32
  0129: n= 18  nnz   162 ->    47
  0130: n= 21  nnz   117 ->    65
  0131: n= 24  nnz   294 ->   113
  0132: n= 27  nnz   250 ->   102
  0133: n= 30  nnz   291 ->   127
  0134: n=  8  nnz     6 ->     2
  0135: n= 12  nnz    21 ->     3
  0136: n= 16 

In [10]:
# Summary table
print(f'{"ID":>6s} {"n":>4s} {"nnz_before":>10s} {"nnz_after":>10s}')
print('-' * 44)
for r in bco_results:
    red = r['nnz_before'] - r['nnz_after']
    print(f'{r["id"]:>6d} {r["n"]:>4d} {r["nnz_before"]:>10d} {r["nnz_after"]:>10d}')

    ID    n nnz_before  nnz_after
--------------------------------------------
   101    8          7          2
   102   14         28          4
   103   20         64          6
   104   50        591         31
   105   21         61          8
   106   42        335         14
   108   42        891         19
   111   27        119         14
   112    5          5          1
   114   11         25          3
   115   28        365         18
   117   42       1003         16
   119   24        101          6
   120    7         11          2
   121   11         30          8
   122   15         63          4
   123   35        582         27
   124   14         78          3
   125    6         11          5
   126    9         25         12
   127   12         30         21
   128   15         79         32
   129   18        162         47
   130   21        117         65
   131   24        294        113
   132   27        250        102
   133   30        291        127
   

## SGE for Circuit Benchmarks

Symmetric Greedy Elimination (SGE) is a fast greedy algorithm for CP decomposition. It iteratively finds the best rank-1 reduction.

We run SGE on the BCO-optimized tensors (ID + 1000).

For small tensors, SGE alone often finds the optimal CP rank.

In [15]:
sge_results = []

for r in bco_results:
    bco_id = r['id'] + 1000
    binary = f"bin/topp{r['n'] // 64 + 1}" # 1,2 or 6
    result = subprocess.run(
        [binary, str(bco_id), '--sge', '--save', '--verify'],
        capture_output=True, text=True
    )
    output = result.stdout

    # Parse CP rank from output
    m_rank = re.search(r'Best rank:\s+(\d+)', output)
    if m_rank:
        cp_rank = int(m_rank.group(1))
        sge_results.append({'id': r['id'],'bco_id': bco_id,'n': r['n'],'nnz': r['nnz_after'],'cp_rank': cp_rank,})
        print(f'  {r["id"]:04d} (bco={bco_id}): n={r["n"]:3d} nnz={r["nnz_after"]:5d} -> CP rank={cp_rank}  [{binary}]')
    else:
        print(f'  {r["id"]:04d}: FAILED')
        print(output)

print(f'\nDone: {len(sge_results)}/{len(bco_results)} tensors processed')


  0101 (bco=1101): n=  8 nnz=    2 -> CP rank=2  [bin/topp1]
  0102 (bco=1102): n= 14 nnz=    4 -> CP rank=4  [bin/topp1]
  0103 (bco=1103): n= 20 nnz=    6 -> CP rank=6  [bin/topp1]
  0104 (bco=1104): n= 50 nnz=   31 -> CP rank=19  [bin/topp1]
  0105 (bco=1105): n= 21 nnz=    8 -> CP rank=8  [bin/topp1]
  0106 (bco=1106): n= 42 nnz=   14 -> CP rank=14  [bin/topp1]
  0108 (bco=1108): n= 42 nnz=   19 -> CP rank=18  [bin/topp1]
  0111 (bco=1111): n= 27 nnz=   14 -> CP rank=11  [bin/topp1]
  0112 (bco=1112): n=  5 nnz=    1 -> CP rank=1  [bin/topp1]
  0114 (bco=1114): n= 11 nnz=    3 -> CP rank=3  [bin/topp1]
  0115 (bco=1115): n= 28 nnz=   18 -> CP rank=12  [bin/topp1]
  0117 (bco=1117): n= 42 nnz=   16 -> CP rank=13  [bin/topp1]
  0119 (bco=1119): n= 24 nnz=    6 -> CP rank=6  [bin/topp1]
  0120 (bco=1120): n=  7 nnz=    2 -> CP rank=2  [bin/topp1]
  0121 (bco=1121): n= 11 nnz=    8 -> CP rank=5  [bin/topp1]
  0122 (bco=1122): n= 15 nnz=    4 -> CP rank=4  [bin/topp1]
  0123 (bco=1123):

In [16]:
# Summary table: BCO + SGE results
print(f'{"ID":>6s} {"n":>4s} {"nnz_orig":>9s} {"nnz_bco":>8s} {"CP rank":>8s}')
print('-' * 39)
for s in sge_results:
    orig = next(r for r in bco_results if r['id'] == s['id'])
    print(f'{s["id"]:>6d} {s["n"]:>4d} {orig["nnz_before"]:>9d} {s["nnz"]:>8d} {s["cp_rank"]:>8d}')

    ID    n  nnz_orig  nnz_bco  CP rank
---------------------------------------
   101    8         7        2        2
   102   14        28        4        4
   103   20        64        6        6
   104   50       591       31       19
   105   21        61        8        8
   106   42       335       14       14
   108   42       891       19       18
   111   27       119       14       11
   112    5         5        1        1
   114   11        25        3        3
   115   28       365       18       12
   117   42      1003       16       13
   119   24       101        6        6
   120    7        11        2        2
   121   11        30        8        5
   122   15        63        4        4
   123   35       582       27       14
   124   14        78        3        3
   125    6        11        5        4
   126    9        25       12        9
   127   12        30       21       16
   128   15        79       32       24
   129   18       162       47       33


## FGS: Flip Graph Search on GF(2^6) Multiplication

For tensors where SGE does not find the optimal CP rank, we use **Flip Graph Search (FGS)** — a randomized local search on the "flip graph" of CP decompositions.

We demonstrate FGS on tensor **0129** (GF(2^6) multiplication, n=18, nnz=51). The best known CP rank is 15.

FGS works by:
1. Starting from an SGE solution
2. Performing random "flips" (rank-preserving moves) to explore the decomposition space
3. Periodically attempting rank reductions

In [ ]:
# SGE + FGS on BCO-optimized tensor
!bin/topp1 129 --sge --fgs -s 1000 -t 8 --plus --save --verify

### FGS on Bilinear Tensors

The `base` variant of the CPD tool works with general (non-symmetric) 3-tensors.
The 05xx series contains GF(2^k) multiplication tensors in bilinear form -- dimensions (k, k, k)
without the symmetric cubic embedding.

BCO does not apply to bilinear tensors (it operates on symmetric cubic tensors only),
so we run SGE+FGS directly on tensor 0529 (GF(2^6) multiplication, 6×6×6).

In [ ]:
# First, generate the known CPD decompositions as starting points:
!python scripts/make_cpd_gf.py

# Compile base variant (bilinear tensors)
!g++ -D VEC_WORDS=1 -D BASE -Ofast -std=c++20 -march=native -s -pthread -I third_party -I src src/cpd.cpp -o bin/base1

# Now run SGE + FGS on tensor 0529
!bin/base1 529 --sge --fgs -s 1000 -t 8 --plus --save --verify

## Verification

The verification script checks that all CP and Waring decompositions in `data/paper/` correctly reconstruct the original tensors.

For each tensor, it:
1. Loads the original tensor and the BCO transform matrix
2. Reconstructs the dense tensor from the CP decomposition
3. Reconstructs the dense tensor from the Waring decomposition
4. Verifies both match the (transformed) original

In [21]:
# numba required
!python scripts/verify_paper_schemes.py

Verifying 88 tensors from data/paper/

    ID     n    CP    CP ok   Waring   War ok
----------------------------------------------
000101     8     2     PASS       13     PASS
000102    14     4     PASS       23     PASS
000103    20     6     PASS       33     PASS
000104    50    16     PASS       83     PASS
000105    21     8     PASS       41     PASS
000106    42    14     PASS       71     PASS
000108    42    17     PASS       77     PASS
000111    27    10     PASS       51     PASS
000112     5     1     PASS        7     PASS
000114    11     3     PASS       17     PASS
000115    28    11     PASS       55     PASS
000117    42    12     PASS       59     PASS
000119    24     6     PASS       37     PASS
000120     7     2     PASS       13     PASS
000121    11     3     PASS       19     PASS
000122    15     4     PASS       25     PASS
000123    35     9     PASS       55     PASS
000124    14     3     PASS       19     PASS
000128    15    13     PASS       59    